Hipótese descrita: PERFIL DO PÚBLICO IDEAL
Dados relacionados: Idade cliente, gênero do cliente, modelo do óculos, tipo de loja, frequência de compra, ticket médio (preço varejo)
Hipótese: Há faixas etárias que concentram maior ticket médio, caracterizando um público-alvo prioritário. 


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
df = pd.read_csv("../../database/dataset gerado/base_vendas_limpa.csv")
df.head()


In [ ]:
from datetime import datetime

df = df.copy()


#verifica se a coluna idade já existe e remove se necessário 
if 'Dim_Cliente.Idade' in df.columns:
    print(":atenção:  Coluna 'Dim_Cliente.Idade' já existe, removendo para recalcular...")
    df = df.drop('Dim_Cliente.Idade', axis=1)


#converte a coluna para datetime (formato mes/dia/ano)
df['Dim_Cliente.Data_Nascimento'] = pd.to_datetime(df['Dim_Cliente.Data_Nascimento'], format='%m/%d/%Y', errors='coerce')


#função para calcular idade precisa
def idade_exata(data_nascimento):
    if pd.isna(data_nascimento):
        return None
    hoje = datetime.now()
    idade = hoje.year - data_nascimento.year


    #verifica se ainda não fez aniversário este ano
    if hoje.month < data_nascimento.month or \
       (hoje.month == data_nascimento.month and hoje.day < data_nascimento.day):
        idade -= 1
    return idade


#criar coluna 'Dim_Cliente.Idade' à direita da coluna de data de nascimento
idades = df['Dim_Cliente.Data_Nascimento'].apply(idade_exata)


#encontrar a posição da coluna de data de nascimento
pos_data_nascimento = df.columns.get_loc('Dim_Cliente.Data_Nascimento')


#insere a nova coluna logo após a coluna de data de nascimento
df.insert(pos_data_nascimento + 1, 'Dim_Cliente.Idade', idades)


#salvar a base com a nova coluna
df.to_csv('../../database/dataset gerado/nova_base_vendas_chillie_com_idade.csv', index=False)


print(":marca_de_verificação_branca: Coluna 'Dim_Cliente.Idade' criada com sucesso!")
print(f":gráfico_de_barras: Total de registros processados: {len(df)}")
print(f":prancheta: Arquivo salvo como: nova_base_vendas_chillie_com_idade.csv")
print("\n" + "="*60)
print(":lupa: VALIDAÇÃO - Primeiras 10 linhas das colunas:")
print("="*60)
print(df[['Dim_Cliente.Data_Nascimento', 'Dim_Cliente.Idade']].head(10))
print("\n" + "="*60)
print(":gráfico_tendência_subindo: RESUMO ESTATÍSTICO DA IDADE:")
print("="*60)
print(df['Dim_Cliente.Idade'].describe())

In [ ]:
AGE_COL = "Dim_Cliente.Idade"
SALES_COL = "Preco_Varejo"  

ticket_medio_idade = df.groupby(AGE_COL)[SALES_COL].mean().reset_index()

ticket_medio_idade


In [ ]:
plt.figure(figsize=(45,20))
sns.barplot(x=AGE_COL, y=SALES_COL, data=ticket_medio_idade, palette="viridis")
plt.title("Ticket Médio por Faixa Etária", fontsize=20)
plt.xlabel("Faixa Etária")
plt.ylabel("Ticket Médio (R$)")
plt.show()


Conclusão: Com base na análise do gráfico - observação que está detalhada na documentação - é possível validar como correta a hipótese 3 gerada, que diz o seguinte: "Há faixas etárias que concentram maior ticket médio, caracterizando um público-alvo prioritário". Esse gráfico mostra que existe um público-alvo aparente e que, esse, diz respeito a pessoas de 67, 79 e 80 anos, ou seja, de acordo com as datas de nascimento registradas e a transformação desses dados em números inteiros (idade).